# Carga de librerías y métodos

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
import torch
from mmseg.apis import init_model, inference_model, show_result_pyplot


In [ ]:
print("CUDA:", torch.cuda.is_available())

In [ ]:
input_dir = '/home/joan_ds/Sandbox/UOC/TFM/trials_imgs'
tmp_dir = 'DeepLabV3Plus-Pytorch/tmp_resized'
output_dir = 'DeepLabV3Plus-Pytorch/output'

In [ ]:
cfg = "/home/joan_ds/mmseg_assets/ocrnet_hr48_4xb2-160k_cityscapes-512x1024.py"
ckpt = "/home/joan_ds/mmseg_assets/ocrnet_hr48_512x1024_160k_cityscapes_20200602_191037-dfbf1b0c.pth"


model = init_model(cfg, ckpt, device='cuda:0')

In [ ]:
CITYSCAPES_ID2COLOR = {
    0: (128, 64, 128),  # road (RGB)
    1: (244, 35, 232),  # sidewalk (RGB)
}

In [ ]:
def filter_mask_ids(ids_map):
    
    h, w = ids_map.shape
    mask_rgb = np.zeros((h, w, 3), dtype=np.uint8)
    present_classes = []
    for class_id, color in CITYSCAPES_ID2COLOR.items():
        m = (ids_map == class_id)
        if m.any():
            mask_rgb[m] = color
            present_classes.append(class_id)
    return mask_rgb, present_classes

def extract_ids_and_conf(result):
    
    ids_map = result.pred_sem_seg.data[0].cpu().numpy().astype(np.uint8)

    conf_map = None
    if hasattr(result, 'seg_logits') and result.seg_logits is not None:
        logits = result.seg_logits
        if hasattr(logits, "data"):   # PixelData -> tensor
            logits = logits.data
        if isinstance(logits, torch.Tensor):
            if logits.dim() == 4:     # [1, C, H, W]
                logits = logits[0]
            if logits.dim() == 3 and logits.shape[0] > 1:  # [C,H,W]
                probs = torch.softmax(logits, dim=0)
                conf_map = probs.max(0).values.cpu().numpy().astype(np.float32)
            elif logits.dim() == 3 and logits.shape[0] == 1:  # [1,H,W]
                conf_map = torch.sigmoid(logits[0]).cpu().numpy().astype(np.float32)
            elif logits.dim() == 2:  # [H,W]
                conf_map = torch.sigmoid(logits).cpu().numpy().astype(np.float32)
    return ids_map, conf_map

def mask_pred_hrnet(model, input_dir,
                    output_dir="hrnet_output", save_ids_conf=True, save_vis=True, alpha=0.5):
    
    os.makedirs(output_dir, exist_ok=True)
    results_lst = []

    files = [f for f in sorted(os.listdir(input_dir)) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    for fname in files:
        stem = os.path.splitext(fname)[0]
        img_path = os.path.join(input_dir, fname)

        result = inference_model(model, img_path)
        ids_map, conf_map = extract_ids_and_conf(result)

        original_bgr = cv2.imread(img_path)
        original_rgb = cv2.cvtColor(original_bgr, cv2.COLOR_BGR2RGB)

        unique = np.unique(ids_map)
        n_classes = int(unique.size)
        road_pct     = float((ids_map == 0).mean())
        sidewalk_pct = float((ids_map == 1).mean())

        road_conf = sidewalk_conf = np.nan
        if conf_map is not None:
            m0 = (ids_map == 0); m1 = (ids_map == 1)
            if m0.any(): road_conf     = float(conf_map[m0].mean())
            if m1.any(): sidewalk_conf = float(conf_map[m1].mean())

        mask_rgb, present_classes = filter_mask_ids(ids_map)
        mask_resized = cv2.resize(mask_rgb, (original_rgb.shape[1], original_rgb.shape[0]),
                                  interpolation=cv2.INTER_NEAREST)
        overlay = cv2.addWeighted(original_rgb, 1 - alpha, mask_resized, alpha, 0)

        if save_ids_conf:
            ids_out  = os.path.join(output_dir, f"{stem}_ids.png")
            cv2.imwrite(ids_out, ids_map)  # uint8 0..18
            if conf_map is not None:
                conf_8 = (np.clip(conf_map, 0, 1) * 255).astype(np.uint8)
                cv2.imwrite(os.path.join(output_dir, f"{stem}_conf.png"), conf_8)

        legend_patches = []
        for class_id in present_classes:
            color_rgb = tuple(np.array(CITYSCAPES_ID2COLOR[class_id]) / 255.0)
            label = "road" if class_id == 0 else "sidewalk"
            legend_patches.append(plt.Rectangle((0,0),1,1, fc=color_rgb, label=label))

        road_conf_txt     = "—" if np.isnan(road_conf) else f"{road_conf:.3f}"
        sidewalk_conf_txt = "—" if np.isnan(sidewalk_conf) else f"{sidewalk_conf:.3f}"
        title = (
            f"{'LINEAR'}: {len(present_classes)} de las 2 clases de interés"
            f"\nnúmero total de clases: {n_classes}"
            f"\nroad: {road_pct*100:.3f}% (conf {road_conf_txt}) "
            f"| sidewalk: {sidewalk_pct*100:.3f}% (conf {sidewalk_conf_txt})"
        )

        fig, axes = plt.subplots(1, 2, figsize=(12, 6))
        axes[0].imshow(original_rgb); axes[0].set_title(f'Original\n{stem}'); axes[0].axis('off')
        axes[1].imshow(overlay); axes[1].set_title(title); axes[1].axis('off')
        if legend_patches:
            fig.legend(handles=legend_patches, loc='lower center', ncol=len(legend_patches))
        plt.tight_layout(); plt.show()

        if save_vis:
            out_path = os.path.join(output_dir, f"{stem}_ocrnet_vis.jpg")
            cv2.imwrite(out_path, cv2.cvtColor(overlay, cv2.COLOR_RGB2BGR))

        results_lst.append({
            'method_name': 'LINEAR',
            'image_name': fname,
            'n_classes': n_classes,
            'road_pct': round(road_pct*100, 3),
            'sidewalk_pct': round(sidewalk_pct*100, 3),
            'road_conf': None if np.isnan(road_conf) else road_conf,
            'sidewalk_conf': None if np.isnan(sidewalk_conf) else sidewalk_conf
        })

        print(f"{stem}: clases={n_classes} | road%={road_pct*100:.3f} | sidewalk%={sidewalk_pct*100:.3f} "
              f"| conf road={road_conf_txt} | conf sidewalk={sidewalk_conf_txt}")

    return results_lst


# Experimentos con imágenes del dataset

In [ ]:
#input_dir = "/home/joan_ds/Sandbox/UOC/TFM/trials_imgs"
results_dataset = mask_pred_hrnet(model=model, input_dir=input_dir, output_dir="ocrnet_output")

# Experimentos con imágenes de los benchmarks

In [ ]:
bdd100k = {'name': 'BDD100K', 'path': '/home/joan_ds/Sandbox/UOC/TFM/benchmarks/BDD100K'}
cityscapes = {'name': 'Cityscapes', 'path': '/home/joan_ds/Sandbox/UOC/TFM/benchmarks/Cityscapes'}
mapillary_vistas = {'name': 'Mapillary Vistas', 'path': '/home/joan_ds/Sandbox/UOC/TFM/benchmarks/Mapillary_Vistas'}

benchmark_lst = [bdd100k, cityscapes, mapillary_vistas]


In [ ]:
results_benchmarks_lst = []

for benchmark in benchmark_lst:
    print (f"Segmentación con el submuestreo del dataset {benchmark['name']}.")

    results_benchmark = mask_pred_hrnet(model=model, input_dir=benchmark['path'], output_dir="hrnet_output")
    results_benchmarks_lst.append(results_benchmark)


# Output

In [ ]:
results_benchmarks_flatten = [dicc for lst in results_benchmarks_lst for dicc in lst]

results_dataset_flatten = [dicc for dicc in results_dataset]

results_dataset_df = pd.DataFrame(results_dataset_flatten)
results_benchmarks_df = pd.DataFrame(results_benchmarks_flatten)

df = pd.concat([results_dataset_df, results_benchmarks_df]).reset_index(drop=True)

df.info()

In [ ]:
df[df.image_name == df.image_name[0]]

In [ ]:
df.to_excel('20250814_results_analysis.xlsx')